In [ ]:
import asyncio
from datetime import date
from src.client.delta_client import DeltaMenuClient
from src.models.requests import MenuQueryRequest

# Test get_menu_by_flight function
async def test_menu_query():
    client = DeltaMenuClient()
    
    request = MenuQueryRequest(
        departure_date=date(2025, 9, 15),
        flight_number=30,
        departure_airport='ATL',
        operating_carrier='DL'
    )
    
    result = await client.get_menu_by_flight(request)
    
    print(f"Success: {result.success}")

    if result.success and result.menu_services:
        print(result)
    else:
        print(f"Error: {result.error_message}")

# Run the test
await test_menu_query()

In [2]:
# Test lookup_flights function
import os
from dotenv import load_dotenv
from src.models.requests import FlightLookupRequest
from datetime import date

# Load environment variables BEFORE importing DeltaMenuClient
load_dotenv(override=True)

from src.client.delta_client import DeltaMenuClient

async def test_flight_lookup():
    # Debug: Check if credentials are loaded
    print(f"Oracle Username: {os.getenv('ORACLE_USERNAME', 'NOT_FOUND')}")
    print(f"Oracle Password: {'SET' if os.getenv('ORACLE_PASSWORD') else 'NOT_FOUND'}")

    client = DeltaMenuClient()
    
    request = FlightLookupRequest(
        departure_date=date(2025, 9, 30),
        departure_airport='ATL',
        arrival_airport='LHR',
        operating_carrier='DL'
    )
    
    result = await client.lookup_flights(request)
    
    print(f'\nFlight Lookup Test:')
    print(f'Success: {result.success}')
    print(f'Route: {result.departure_airport} -> {result.arrival_airport}')
    print(f'Date: {result.departure_date}')
    
    if result.success and result.flights:
        print(f'Found {len(result.flights)} flights:')
        for flight in result.flights:
            print(f'  DL{flight.flight_number}: {flight.departure_time} -> {flight.arrival_time}')
    else:
        print(f'Error: {result.error_message}')
    
    await client.close()

# Run flight lookup test
await test_flight_lookup()

2025-09-23 12:32:14,897 - src.client.delta_client - __init__ - INFO - Initializing DeltaMenuClient


Oracle Username: 940985
Oracle Password: SET


2025-09-23 12:32:16,946 - src.client.delta_client - lookup_flights - INFO - Looking up flights from ATL to LHR on 2025-09-30
2025-09-23 12:32:23,864 - src.database.flight_repository - lookup_flights - INFO - Querying flights: ATL to LHR on 2025-09-30
2025-09-23 12:32:24,673 - src.client.delta_client - close - INFO - Closing DeltaMenuClient



Flight Lookup Test:
Success: True
Route: ATL -> LHR
Date: 2025-09-30
Found 3 flights:
  DL30: 21:25 -> 05:45
  DL32: 01:55 -> 10:10
  DL36: 03:59 -> 12:30


In [ ]:
# Test check_menu_availability function
from src.models.menu import FlightLeg
from datetime import date
from src.client.delta_client import DeltaMenuClient
from src.models.requests import MenuQueryRequest

async def test_menu_availability():
    client = DeltaMenuClient()
    
    flight_legs = [
        FlightLeg(
            operating_carrier_code='DL',
            flight_num=30,
            flight_departure_airport_code='ATL',
            departure_local_date='2025-08-26'
        )
    ]
    
    result = await client.check_menu_availability(flight_legs)
    
    print(f"Success: {result.success}")
    if result.success:
        print(result)
    else:
        print(f"Error: {result.error_message}")

# Run the test
await test_menu_availability()